# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import torch
from torchvision import transforms,datasets, models
from torch import nn,optim
import json
import time

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
# FLOWERS
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# # FLOWERS
# # Define your transforms for the training, validation, and testing sets
# training_transforms = transforms.Compose([transforms.RandomRotation(30),
#                                           transforms.RandomResizedCrop(224),
#                                           transforms.RandomHorizontalFlip(),
#                                           transforms.ToTensor(),
#                                           transforms.Normalize([0.485, 0.456, 0.406],
#                                                                [0.229, 0.224, 0.225])])

# validation_transforms = transforms.Compose([transforms.Resize(255),
#                                           transforms.CenterCrop(224),
#                                           transforms.ToTensor(),
#                                           transforms.Normalize([0.485, 0.456, 0.406],
#                                                                [0.229, 0.224, 0.225])])

# testing_transforms = transforms.Compose([transforms.Resize(255),
#                                           transforms.CenterCrop(224),
#                                           transforms.ToTensor(),
#                                           transforms.Normalize([0.485, 0.456, 0.406],
#                                                                [0.229, 0.224, 0.225])])

# # Load the datasets with ImageFolder
# training_dataset = datasets.ImageFolder(train_dir, transform=training_transforms)
# validation_dataset = datasets.ImageFolder(valid_dir, transform=validation_transforms)
# testing_dataset = datasets.ImageFolder(test_dir, transform=testing_transforms)

# # Using the image datasets and the trainforms, define the dataloaders
# training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=100, shuffle=True)
# validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=100, shuffle=True)
# testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=100)
 

In [3]:
#CATS AND DOGS
# Define transforms for the training data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

# Define transforms for the testing data
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

# Define transforms for the extra data
extra_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])


# Upload training data
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)

# Upload testing data
test_data = datasets.ImageFolder(valid_dir, transform=test_transforms)

# Upload extra data
extra_data = datasets.ImageFolder(test_dir, transform=extra_transforms)

# Create training DataLoader, define batch size and shuffle configuration ON
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

# Create testing DataLoader, define batch size and shuffle configuration OFF
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

# Create extra DataLoader, define batch size and shuffle configuration OFF
extraloader = torch.utils.data.DataLoader(extra_data, batch_size=64)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
# FLOWERS
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
#     print(len(cat_to_name)) 

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

∞∞**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [6]:
# FLOWERS
# # Building the Network

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # load pre-trained model

# model = models.vgg16(pretrained=True)

# for param in model.parameters():
#     param.requires_grad = False
    
# # create classifier layer and replace in model
# model.classifier = nn.Sequential(nn.Linear(25088,len(cat_to_name)),
#                                  nn.LogSoftmax(dim=1))

# criterion = nn.NLLLoss()

# optimizer = optim.Adam(model.classifier.parameters(), lr=0.005)

# model.to(device);

In [7]:
# CATS AND DOGS
# Make code agnostic to use CPU or GPU(if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretreined neural network
model = models.densenet121(pretrained=True)

# We don't want to backpropagate to the trained layers, so we freeze them
for param in model.parameters():
    param.requires_grad = False
    
# create the new last layer to be trained
classifier = nn.Sequential(nn.Linear(1024,256),
                          nn.ReLU(),
                          nn.Dropout(0.2),                           
                          nn.Linear(256,len(cat_to_name)),
                          nn.LogSoftmax(dim=1))

# replace the original trained neural network classifier layer with the newly create one
model.classifier = classifier

# Choose your loss function
criterion = nn.NLLLoss()

# TODO: Check if this works: optmizer = optim.Adam(parameters)
optmizer = optim.Adam(model.classifier.parameters(), lr=0.003)

# TODO: Check if this works: model.to(device)
model.to(device);

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 81032159.51it/s]


In [8]:
# # FLOWERS
# # Training the Network
# epochs=3
# batch = 0
# validation_step = 5
# running_loss = 0
# for epoch in range(epochs):    
#     for images,labels in training_dataloader:
#         images,labels = images.to(device),labels.to(device)
#         batch += 1
#         optimizer.zero_grad()
        
#         output = model.forward(images)
#         loss = criterion(output,labels)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
        
#         if batch % validation_step == 0:           
#             model.eval()
#             validation_loss = 0
#             validation_accuracy = 0

#             with torch.no_grad():
#                 for valid_images,valid_labels in validation_dataloader:
#                     valid_images,valid_labels = valid_images.to(device),valid_labels.to(device)

#                     valid_output = model.forward(valid_images)
#                     valid_loss = criterion(valid_output,valid_labels)
#                     validation_loss += valid_loss.item()

#                     top_p,top_class = torch.exp(valid_output).topk(1,dim=1)
#                     valid_equals = top_class == valid_labels.view(*top_class.shape)
#                     validation_accuracy = torch.mean(valid_equals.type(torch.FloatTensor)).item()
                    
#             print(f"Epoch {epoch+1}/{epochs}.. "
#                   f"Train loss: {running_loss/validation_step:.3f}.. "
#                   f"Validation loss: {validation_loss/len(validation_dataloader):.3f}.. "
#                   f"Validation accuracy: {validation_accuracy/len(validation_dataloader):.3f}\n")

#             validation_loss = 0
#             model.train()
#             running_loss = 0

In [9]:
# CATS AND DOGS
# !! Let's start training !!

# Define number of Epochs
# TODO: test with other epochs
epochs = 1

# Create variables to track number of training steps and running training loss
step = 0
running_loss = 0

# Create variable to print running loss every X steps
print_every = 5

# Train model for "epochs" epochs
for e in range(epochs):
    # get the next batch of training images
    for images, labels in trainloader:
        step += 1
        # Move input and label tensors to the default device
        images, labels = images.to(device), labels.to(device)
        
        # make sure past gradients won't influence this training step
        optmizer.zero_grad()
        
#         do one training step
        output = model.forward(images)
        loss = criterion(output,labels)
        loss.backward()
        optmizer.step()

        # update running loss
        running_loss += loss.item()
        
        # Check if it's time to print model running Loss and Accuracy
        if step % print_every == 0:
            test_losses = 0
            accuracy = 0
            
            # make sure we are in "eval" mode, i.e. turn off drop out
            model.eval()
            
            with torch.no_grad():
                for test_images, test_labels in testloader:
                    # Move input and label tensors to the default device
                    test_images, test_labels = test_images.to(device), test_labels.to(device)

                    # Uses network to calculate output
                    test_output = model.forward(test_images)

                    # Calculate loss produced by network
                    test_loss = criterion(test_output,test_labels)
                    # update testing running loss
                    test_losses += test_loss.item()
            
                    # Calculate accuracy
                    test_output_prob = torch.exp(test_output)
                    top_test_output_prob,top_test_output_class = test_output_prob.topk(1, dim =1)
                    equals = top_test_output_class == test_labels.view(*top_test_output_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            print(f"Epoch {e+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_losses/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")

            # remember to set mode to "train" after testing
            model.train()
            
            # reset training running loss cause it's gonna start a new epoch after this
            running_loss = 0
            
            
        
        


Epoch 1/1.. Train loss: 4.647.. Test loss: 4.475.. Test accuracy: 0.080
Epoch 1/1.. Train loss: 4.346.. Test loss: 4.324.. Test accuracy: 0.091
Epoch 1/1.. Train loss: 4.261.. Test loss: 4.119.. Test accuracy: 0.098
Epoch 1/1.. Train loss: 4.209.. Test loss: 3.909.. Test accuracy: 0.164
Epoch 1/1.. Train loss: 3.998.. Test loss: 3.715.. Test accuracy: 0.215
Epoch 1/1.. Train loss: 3.943.. Test loss: 3.481.. Test accuracy: 0.217
Epoch 1/1.. Train loss: 3.530.. Test loss: 3.213.. Test accuracy: 0.262
Epoch 1/1.. Train loss: 3.383.. Test loss: 2.920.. Test accuracy: 0.390
Epoch 1/1.. Train loss: 3.036.. Test loss: 2.668.. Test accuracy: 0.437
Epoch 1/1.. Train loss: 3.112.. Test loss: 2.481.. Test accuracy: 0.446
Epoch 1/1.. Train loss: 2.707.. Test loss: 2.244.. Test accuracy: 0.490
Epoch 1/1.. Train loss: 2.430.. Test loss: 2.046.. Test accuracy: 0.514
Epoch 1/1.. Train loss: 2.451.. Test loss: 1.865.. Test accuracy: 0.620
Epoch 1/1.. Train loss: 2.241.. Test loss: 1.742.. Test accuracy

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [10]:
# # FLOWERS
# # Do validation on the test set
# model.eval()
# testing_losses = 0
# testing_accuracy = 0
# with torch.no_grad():
#     for testing_images,testing_labels in testing_dataloader:
#         testing_images,testing_labels = testing_images.to(device),testing_labels.to(device)

#         testing_output = model.forward(testing_images)
#         testing_loss = criterion(testing_output,testing_labels)
#         testing_losses += testing_loss.item()

#         top_p,top_class = torch.exp(testing_output).topk(1,dim=1)
#         testing_equals = top_class == testing_labels.view(*top_class.shape)
#         testing_accuracy = torch.mean(testing_equals.type(torch.FloatTensor)).item()

# print(f"Test loss: {testing_losses/len(testing_dataloader):.3f}.. "
#       f"Test accuracy: {testing_accuracy/len(testing_dataloader):.3f}\n")

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model = models.densenet121(pretrained=True)
classifier = nn.Sequential(nn.Linear(1024,256),
                          nn.ReLU(),
                          nn.Dropout(0.2),                           
                          nn.Linear(256,len(cat_to_name)),
                          nn.LogSoftmax(dim=1))
model.classifier = classifier
for param in model.parameters():
    param.requires_grad = False

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 88160675.39it/s]


In [7]:
state_dict = torch.load('densenet121_optim_and_model_state_dict_checkpoint.pth')
model.load_state_dict(state_dict['model_state_dict'])

In [15]:
# Choose your loss function
criterion = nn.NLLLoss()
model.to(device);

In [17]:
extra_losses = 0
extra_accuracy = 0

# make sure we are in "eval" mode, i.e. turn off drop out
model.eval()

with torch.no_grad():
    for test_images, test_labels in extraloader:
        # Move input and label tensors to the default device
        test_images, test_labels = test_images.to(device), test_labels.to(device)

        # Uses network to calculate output
        test_output = model.forward(test_images)

        # Calculate loss produced by network
        test_loss = criterion(test_output,test_labels)
        # update testing running loss
        extra_losses += test_loss.item()

        # Calculate accuracy
        test_output_prob = torch.exp(test_output)
        top_test_output_prob,top_test_output_class = test_output_prob.topk(1, dim =1)
        equals = top_test_output_class == test_labels.view(*top_test_output_class.shape)
        extra_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {extra_losses/len(extraloader):.3f}.. "
      f"Test accuracy: {extra_accuracy/len(extraloader):.3f}")

Test loss: 1.234.. Test accuracy: 0.704


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [11]:
# # FLOWERS
# # Save the checkpoint 
# model.class_to_idx = training_dataset.class_to_idx

# checkpoint = {'optimizer_state_dict': optimizer.state_dict(),
#               'model_state_dict': model.state_dict()}

# torch.save(checkpoint, 'th_model_state_dict_checkpoint.pth')


In [17]:
# # CATS AND DOGS
# # Save the checkpoint 
# model.class_to_idx = train_data.class_to_idx

# checkpoint = {'optimizer_state_dict': optmizer.state_dict(),
#               'model_state_dict': model.state_dict()}

# torch.save(checkpoint, 'densenet121_optim_and_model_state_dict_checkpoint.pth')


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [20]:
# Write a function that loads a checkpoint and rebuilds the model
state_dict = torch.load('densenet121_optim_and_model_state_dict_checkpoint.pth')
print(state_dict.keys())
model.load_state_dict(state_dict['model_state_dict'])

dict_keys(['optimizer_state_dict', 'model_state_dict'])


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [13]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [14]:
def imshow(image, ax=None, title=None):
Imshow for Tensor.
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

IndentationError: expected an indented block (<ipython-input-14-6fcbc89ce119>, line 2)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [5]:
import torch 
state_dict = torch.load('FinalProject/training_checkpoints/resnet_state_dict_checkpoint.pth', map_location='cpu')
print(state_dict)

{'arch': 'resnet', 'output_units': 102, 'hidden_units': 256, 'batch_size': 128, 'epochs': 5, 'learning_rate': 0.003, 'checkpoint_save_dir': 'training_checkpoints/', 'class_to_idx': {'1': 0, '10': 1, '100': 2, '101': 3, '102': 4, '11': 5, '12': 6, '13': 7, '14': 8, '15': 9, '16': 10, '17': 11, '18': 12, '19': 13, '2': 14, '20': 15, '21': 16, '22': 17, '23': 18, '24': 19, '25': 20, '26': 21, '27': 22, '28': 23, '29': 24, '3': 25, '30': 26, '31': 27, '32': 28, '33': 29, '34': 30, '35': 31, '36': 32, '37': 33, '38': 34, '39': 35, '4': 36, '40': 37, '41': 38, '42': 39, '43': 40, '44': 41, '45': 42, '46': 43, '47': 44, '48': 45, '49': 46, '5': 47, '50': 48, '51': 49, '52': 50, '53': 51, '54': 52, '55': 53, '56': 54, '57': 55, '58': 56, '59': 57, '6': 58, '60': 59, '61': 60, '62': 61, '63': 62, '64': 63, '65': 64, '66': 65, '67': 66, '68': 67, '69': 68, '7': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '8': 80, '80': 81, '81': 82, '82